In [ ]:
import json
import pandas as pd
import requests
from simple_salesforce import Salesforce, SalesforceLogin, SFType

In [7]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

In [8]:
results = sf.query_all(
    """
   select Id, IdentificationId__pc from Account 
    """
)
df = pd.DataFrame(results["records"])
df.shape
Pacientes_SF = df[["Id", "IdentificationId__pc"]]
Pacientes_SF.shape

(1542676, 2)

In [9]:
# Recuperar empresas ID de salesforce
results = sf.query_all(
    """
   select Id, Name, Holding__c ,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,Empleador__c, ParentId, IsPersonAccount,poliza__c  from account where ispersonaccount = false 
    """
)
df = pd.DataFrame(results["records"])
df.shape
Empresas_account_SF = df[["Id", "Name", "Pa_s_del_Paciente__c", "Campa_a__c","AccountNumber",'Id_Empresa__c','Empleador__c','ParentId','IsPersonAccount','poliza__c']]
Empresas_account_SF.head()



,Id,Name,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,Empleador__c,ParentId,IsPersonAccount,poliza__c
0,0018c00002eTcCQAA0,Vida Camara Holding Vida Camara Cintegral Sist...,a1U8c000005dAtnEAE,Vida Camara,None,EMP879,None,0018c00002axQWuAAM,False,None
1,0018c00002eTcCRAA0,Vida Camara Holding Vida Camara Civil Agro S.A.,a1U8c000005dAtnEAE,Vida Camara,None,EMP891,None,0018c00002axQWuAAM,False,None
2,0018c00002eTcCSAA0,Vida Camara Holding Vida Camara Civilplas S.A.,a1U8c000005dAtnEAE,Vida Camara,None,EMP893,None,0018c00002axQWuAAM,False,None
3,0018c00002eTcCTAA0,Vida Camara Holding Vida Camara Cj Traffic Chi...,a1U8c000005dAtnEAE,Vida Camara,None,EMP894,None,0018c00002axQWuAAM,False,None
4,0018c00002eTcCUAA0,Vida Camara Holding Vida Camara Clinda Ltda.,a1U8c000005dAtnEAE,Vida Camara,None,EMP895,None,0018c00002axQWuAAM,False,None


In [10]:
#traer empresas de accouint para unir con empresas sf
results = sf.query_all(
    """
   select id,Name,Nombre_Empresa__c,Empresa__c, Empresa_Padre__c from Programas_por_Empresas__c
    """
)
df = pd.DataFrame(results["records"])
df.shape
Empresas_SF = df[["Id", "Nombre_Empresa__c", "Name", "Empresa__c","Empresa_Padre__c"]]
Empresas_SF.head()


,Id,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,a1W8c000007vDN8EAM,Metlife Holding Metlife Laboratorio Maver Ltda.,A-0000,0018c00002axQXrAAM,MetLife
1,a1W8c000007vDN9EAM,Aguas Del Valle S.A.,A-0001,0018c00002axQXsAAM,MetLife
2,a1W8c000007vDNAEA2,Metlife Holding Metlife Berliam Spa.,A-0002,0018c00002axQXtAAM,MetLife
3,a1W8c000007vDNBEA2,Besalco Maquinarias S.A.,A-0003,0018c00002axQXuAAM,MetLife
4,a1W8c000007vDNCEA2,Carlos Escarate Y Cia Ltda.,A-0004,0018c00002axQXvAAM,MetLife


In [11]:
empresas = pd.merge(
    left=Empresas_account_SF,
    right=Empresas_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
empresas.head()

,Id_x,Name_x,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,Empleador__c,ParentId,IsPersonAccount,poliza__c,Id_y,Nombre_Empresa__c,Name_y,Empresa__c,Empresa_Padre__c
0,0018c00002eTcCQAA0,Vida Camara Holding Vida Camara Cintegral Sist...,a1U8c000005dAtnEAE,Vida Camara,None,EMP879,None,0018c00002axQWuAAM,False,None,a1W8c0000083l2BEAQ,Vida Camara Holding Vida Camara Cintegral Sist...,A-2357,0018c00002eTcCQAA0,Vida Camara
1,0018c00002eTcCRAA0,Vida Camara Holding Vida Camara Civil Agro S.A.,a1U8c000005dAtnEAE,Vida Camara,None,EMP891,None,0018c00002axQWuAAM,False,None,a1W8c0000083l2CEAQ,Vida Camara Holding Vida Camara Civil Agro S.A.,A-2358,0018c00002eTcCRAA0,Vida Camara
2,0018c00002eTcCSAA0,Vida Camara Holding Vida Camara Civilplas S.A.,a1U8c000005dAtnEAE,Vida Camara,None,EMP893,None,0018c00002axQWuAAM,False,None,a1W8c0000083l2DEAQ,Vida Camara Holding Vida Camara Civilplas S.A.,A-2359,0018c00002eTcCSAA0,Vida Camara
3,0018c00002eTcCTAA0,Vida Camara Holding Vida Camara Cj Traffic Chi...,a1U8c000005dAtnEAE,Vida Camara,None,EMP894,None,0018c00002axQWuAAM,False,None,a1W8c0000083l2EEAQ,Vida Camara Holding Vida Camara Cj Traffic Chi...,A-2360,0018c00002eTcCTAA0,Vida Camara
4,0018c00002eTcCUAA0,Vida Camara Holding Vida Camara Clinda Ltda.,a1U8c000005dAtnEAE,Vida Camara,None,EMP895,None,0018c00002axQWuAAM,False,None,a1W8c0000083l2FEAQ,Vida Camara Holding Vida Camara Clinda Ltda.,A-2361,0018c00002eTcCUAA0,Vida Camara


In [12]:
empresas.shape

(7930, 15)

In [13]:
empresas.to_excel("F:/Empresas SF/reportes/emresas_.xlsx",index=False)

In [14]:

# Relacion pacientes y empresas
results = sf.query_all(
    """
  select Id,Paciente__c,Programas_por_Empresas__c from Paciente_Programas__c 
    """
)
df = pd.DataFrame(results)
df.shape
Pacientes_empresas = pd.DataFrame(results["records"])
Pacientes_empresas = Pacientes_empresas[
    ["Paciente__c", "Programas_por_Empresas__c"]
]
Pacientes_empresas.head()

,Paciente__c,Programas_por_Empresas__c
0,0018c00002amLbpAAE,a1W8c000007vDPvEAM
1,0018c00002amLbqAAE,a1W8c000007vDSpEAM
2,0018c00002amLbrAAE,a1W8c000007vDSpEAM
3,0018c00002amLbsAAE,a1W8c000007vDSpEAM
4,0018c00002amLc2AAE,a1W8c000007vDSpEAM


In [15]:
#une Pacientes y el que hace de union Pacientes_empresas
pacientes_y_union = pd.merge(
    left=Pacientes_SF,
    right=Pacientes_empresas,
    how="left",
    left_on="Id",
    right_on="Paciente__c",
)
pacientes_y_union.head()

,Id,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c
0,0018c00002amLTMAA2,173805004,0018c00002amLTMAA2,a1W8c0000083icPEAQ
1,0018c00002amLUGAA2,153338949,NaN,NaN
2,0018c00002amLUHAA2,177735213,0018c00002amLUHAA2,a1W8c0000083j8wEAA
3,0018c00002amLUIAA2,169332924,0018c00002amLUIAA2,a1W8c0000083laZEAQ
4,0018c00002amLUJAA2,172957692,0018c00002amLUJAA2,a1W8c0000083iWhEAI


In [16]:
pacientes_y_union.shape

(1653164, 4)

In [17]:
pacientes_y_empresas_final = pd.merge(
    left=pacientes_y_union,
    right=Empresas_SF,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id",
)
pacientes_y_empresas_final.head()

,Id_x,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c,Id_y,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,0018c00002amLTMAA2,173805004,0018c00002amLTMAA2,a1W8c0000083icPEAQ,a1W8c0000083icPEAQ,Vulco S.A.,A-1075,0018c00002cXJZeAAO,MetLife
1,0018c00002amLUGAA2,153338949,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0018c00002amLUHAA2,177735213,0018c00002amLUHAA2,a1W8c0000083j8wEAA,a1W8c0000083j8wEAA,Farmacias Cruz Verde,A-2322,0018c00002cXclVAAS,Farmacias Cruz Verde
3,0018c00002amLUIAA2,169332924,0018c00002amLUIAA2,a1W8c0000083laZEAQ,a1W8c0000083laZEAQ,EY - EY Audit SpA,A-4489,0018c00002eTboVAAS,EY
4,0018c00002amLUJAA2,172957692,0018c00002amLUJAA2,a1W8c0000083iWhEAI,a1W8c0000083iWhEAI,Minera Abra Holding Minera Abra Minera Abra,A-0721,0018c00002cXJTrAAO,Minera Abra


In [18]:
pacientes_y_empresas_final.shape

(1653164, 9)

In [36]:
nuevos = ("F:/Empresas SF/reportes/activos.xlsx")
Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

,rut1,nombres,apellidos,Relacion_con_Titular,Rut_Titular,Activo,Empleador,rut
0,27275521-3,Tomas,Agustin Herrera Baez,Hijo/a,16320950-0,1,Cenim,272755213
1,11486141-3,Carlos,Neil Valencia Orrego,Titular,11486141-3,1,Cenim,114861413
2,8874614-7,Luis,Hernan Castro Camus,Titular,8874614-7,1,Cenim,88746147
3,11318307-1,Jessica,Allendes Valdebenitos,Cónyuge,9702560-6,1,Cenim,113183071
4,14305806-9,Eduardo,Hernandez Reyes,Titular,14305806-9,1,Cenim,143058069


In [37]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
#Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.upper()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].astype(str)
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.strip()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["IdentificationId__pc"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_6688\1291638168.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_6688\1291638168.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [39]:
Pacientes_Archivo.head(20)

,rut1,nombres,apellidos,Relacion_con_Titular,Rut_Titular,Activo,Empleador,rut,RutEstaoNo
0,27275521-3,Tomas,Agustin Herrera Baez,Hijo/a,16320950-0,1,Cenim,272755213,True
1,11486141-3,Carlos,Neil Valencia Orrego,Titular,11486141-3,1,Cenim,114861413,True
2,8874614-7,Luis,Hernan Castro Camus,Titular,8874614-7,1,Cenim,88746147,True
3,11318307-1,Jessica,Allendes Valdebenitos,Cónyuge,9702560-6,1,Cenim,113183071,True
4,14305806-9,Eduardo,Hernandez Reyes,Titular,14305806-9,1,Cenim,143058069,True
5,18562360-2,Karen,Andrea Castro Aguilera,Cónyuge,16917678-7,1,Cenim,185623602,True
6,22198154-5,Sofia,A Lamilla Gamboa,Hijo/a,16183085-2,1,Cenim,221981545,True
7,16183085-2,Francisco,Lamilla Estrada,Titular,16183085-2,1,Cenim,161830852,True
8,15063473-3,David,Marco Vera Estay,Titular,15063473-3,1,Cenim,150634733,True
9,16402593-4,Daniela,Graciela Tapia Manzano,Cónyuge,15066310-5,1,Cenim,164025934,True


In [40]:
Pacientes_con_ID = pd.merge(
    left=Pacientes_Archivo,
    right=pacientes_y_empresas_final,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc",
)
Pacientes_con_ID.head()


,rut1,nombres,apellidos,Relacion_con_Titular,Rut_Titular,Activo,Empleador,rut,RutEstaoNo,Id_x,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c,Id_y,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,27275521-3,Tomas,Agustin Herrera Baez,Hijo/a,16320950-0,1,Cenim,272755213,True,0018c00002axcTXAAY,272755213,0018c00002axcTXAAY,a1W8c000007vDVLEA2,a1W8c000007vDVLEA2,Bci Holding Bci Cenim,A-0509,0018c00002axQgdAAE,BCI
1,11486141-3,Carlos,Neil Valencia Orrego,Titular,11486141-3,1,Cenim,114861413,True,0018c00002axcTYAAY,114861413,0018c00002axcTYAAY,a1W8c000007vDVLEA2,a1W8c000007vDVLEA2,Bci Holding Bci Cenim,A-0509,0018c00002axQgdAAE,BCI
2,8874614-7,Luis,Hernan Castro Camus,Titular,8874614-7,1,Cenim,88746147,True,0018c00002axcTZAAY,88746147,0018c00002axcTZAAY,a1W8c000007vDVLEA2,a1W8c000007vDVLEA2,Bci Holding Bci Cenim,A-0509,0018c00002axQgdAAE,BCI
3,11318307-1,Jessica,Allendes Valdebenitos,Cónyuge,9702560-6,1,Cenim,113183071,True,0018c00002axcTaAAI,113183071,0018c00002axcTaAAI,a1W8c000007vDVLEA2,a1W8c000007vDVLEA2,Bci Holding Bci Cenim,A-0509,0018c00002axQgdAAE,BCI
4,14305806-9,Eduardo,Hernandez Reyes,Titular,14305806-9,1,Cenim,143058069,True,0018c00002axcTbAAI,143058069,0018c00002axcTbAAI,a1W8c000007vDVLEA2,a1W8c000007vDVLEA2,Bci Holding Bci Cenim,A-0509,0018c00002axQgdAAE,BCI


In [41]:
Pacientes_con_ID.to_excel(
    "F:/Empresas SF/reportes/Pacientes_y_empresas_nuevas.xlsx", index=False
)


In [43]:
Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]
Final_pacientes_y_empresa = Final_pacientes_y_empresa[
    Final_pacientes_y_empresa["RutEstaoNo"] == False
]

if ("nombres" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["FirstName"] = Final_pacientes_y_empresa["nombres"]
if ("apellidos" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["LastName"] = Final_pacientes_y_empresa["apellidos"]
if ("rut" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Final_pacientes_y_empresa["rut"]
if ("email" in Final_pacientes_y_empresa.columns) == True:
    Final_pacientes_y_empresa["email"] = Final_pacientes_y_empresa["email"].str.lower()
    Subir_pacientes["PersonEmail"] = Final_pacientes_y_empresa["email"]
if ("nacimiento" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Final_pacientes_y_empresa["nacimiento"]
if ("sexo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Final_pacientes_y_empresa["sexo"]
if ("direccion" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
if ("cargo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonTitle"] = Final_pacientes_y_empresa["cargo"]
if ("division" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonDepartment"] = Final_pacientes_y_empresa["division"]

Subir_pacientes["BillingCountry"] = "Chile"
Subir_pacientes["Identification_Type__c"] = "RUT"
Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"


Subir_pacientes.to_excel(
     "F:/Empresas SF/reportes/account.xlsx", index=False
)
Subir_pacientes.head()


,FirstName,LastName,IdentificationId__pc,BillingCountry,Identification_Type__c,RecordTypeId
6545,Rodrigo Alejandro,Rodrigo Alejandro Cabello Diaz ...,15018701K,Chile,RUT,0128c000002Byp4AAC
6700,Rocco Calivar,Rocco Calivar Juan Alejandro ...,13759501K,Chile,RUT,0128c000002Byp4AAC
6704,Romero Urriaga,Romero Urriaga Patricio Jalem ...,15557027K,Chile,RUT,0128c000002Byp4AAC
6834,Elizabeth De Las Mercedes,Elizabeth De Las Mercedes Jara Muðoz ...,13538741K,Chile,RUT,0128c000002Byp4AAC
7034,Luis Raymundo,Luis Raymundo Esparza Rodriguez ...,12549787K,Chile,RUT,0128c000002Byp4AAC


In [44]:
nuevos = ("F:/Empresas SF/reportes/activos.xlsx")
Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

pacientes_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc",
)
pacientes_id.head()
pacientes_id.to_excel(
     "F:/Empresas SF/reportes/archivo_id.xlsx", index=False
)